# Twitter Accounts to Follow

@OpenANC should generally follow all current commissioners and candidates.

In [1]:
import os
os.chdir('../')

In [2]:
import time
import tweepy
import random
import pandas as pd
from tqdm import tqdm

from scripts.data_transformations import list_commissioners

consumer_key = os.environ.get('TWITTER_API_KEY')
consumer_secret = os.environ.get('TWITTER_API_KEY_SECRET')
access_key = os.environ.get('TWITTER_ACCESS_TOKEN')
access_secret = os.environ.get('TWITTER_ACCESS_TOKEN_SECRET')

In [3]:
ancs = pd.read_csv('data/ancs.csv')
people = pd.read_csv('data/people.csv')
candidates = pd.read_csv('data/candidates.csv')
candidate_statuses = pd.read_csv('data/candidate_statuses.csv')
commissioners = list_commissioners(status='current')

people_candidates = pd.merge(people, candidates, how='inner', on='person_id')
people_candidates_status = pd.merge(people_candidates, candidate_statuses, how='inner', on='candidate_status')
people_candidates_active = people_candidates_status[
    (people_candidates_status['count_as_candidate'])
    & (people_candidates_status['election_year'] == 2022)
].copy()

people_commissioners = pd.merge(people, commissioners, how='inner', on='person_id')

# Group all twitter links for active candidates, current commissioners, and commissions in one list
twitter_links = pd.concat([
    people_commissioners.twitter_link
    , people_candidates_active.twitter_link
    , ancs.twitter_link
])

In [4]:
twitter_handles = (
    twitter_links[twitter_links.notnull()]
    .str.replace('https://twitter.com/', '')
    .str.lower()
)

list_of_handles = sorted(twitter_handles.unique())
len(list_of_handles)

237

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [6]:
%%time

openanc_follows = []
for page_result in tweepy.Cursor(api.friends, screen_name='OpenANC').pages():
    
    for user in page_result:
        openanc_follows += [user.screen_name.lower()]

    time.sleep(5)

CPU times: user 441 ms, sys: 46.4 ms, total: 488 ms
Wall time: 1min 10s


In [7]:
should_follow = [t for t in list_of_handles if t not in openanc_follows]
len(should_follow)

2

In [18]:
# for f in should_follow:
#     print('https://twitter.com/' + f)

List of twitter accounts that OpenANC follows that are NOT in the database. 

In [17]:
known_non_commissioners = [
    'bradyhunch'
    , 'mapbox'
    , 'vote4dc'
    , 'ancsof'
    , 'opendatadc'
    , 'kenyanmcduffie'
    , 'brianneknadeau'
    , 'cmlewisgeorgew4'
    , 'chmnmendelson'
    , 'anitabondsdc'
    , 'trayonwhite'    
    , 'marycheh'
    , 'charlesallen'
    , 'planitmichael'
    , 'heycaseywattsup'
    , '_matthewthomas'
    , 'infinitebuffalo'
    , 'revwendy3'
    , 'allinsea'
    , 'lorenzoward7dc'
]

followed_but_not_in_db = [
    t for t in openanc_follows if (t not in list_of_handles and t not in known_non_commissioners)
]

for f in followed_but_not_in_db:
    print('https://twitter.com/' + f)

https://twitter.com/johnzottoli
https://twitter.com/cmoney_htx
https://twitter.com/snikiddyschmeow
